In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
df['age_scaled'] = df.age / 100

In [4]:
X = df[['age_scaled', 'affordibility']].values
Y = df.bought_insurance.values
X.shape, Y.shape

((28, 2), (28,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=1)
X_train.shape, X_test.shape

((22, 2), (6, 2))

# Making a Model from Scratch

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
def binary_crossEntropy(y_true, y_pred):
    epsilon = 1e-15  # to avoid log(0)
    y_pred = [max(i, epsilon) for i in y_pred]
    y_pred = [min(i, 1-epsilon) for i in y_pred]

    y_pred = np.array(y_pred)
    
    return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [8]:
class myNN():
    def __init__(self):
        pass
        
    def gradient_descent(self, x1, x2, y_true, epochs, loss_threshold):
        w1 = w2 = 1
        bias = 0
        learning_rate = 0.5
        n = len(y_true)
        
        for i in range(1, epochs + 1):
            weighted_sum = w1*x1 + w2*x2 + bias
            y_pred = sigmoid(weighted_sum)  # activation function
        
            loss = binary_crossEntropy(y_true, y_pred)  # loss function

            print('epochs={:<5}, w1={:<25}, w2={:<25}, bias={:<25}, loss={:<25}'.format(i, w1, w2, bias, loss))
            
            if loss <= loss_threshold:
                break
            
            # Compute gradients (derivatives of the loss function to weights and bias
            error = np.transpose(y_pred - y_true)  # chuyển vị thành ma trận 1x1
            d_w1 = (1/n) * np.dot(error, x1)  # 
            d_w2 = (1/n) * np.dot(error, x2)
            d_bias = np.mean(error)
            # Update weights and bias
            w1 = w1 - learning_rate * d_w1
            w2 = w2 - learning_rate * d_w2
            bias = bias - learning_rate * d_bias
                
        return w1, w2, bias, loss
        
    
    def fit(self, X_train, y_train, epochs, loss_threshold):
        self.w1, self.w2, self.bias, self.loss = self.gradient_descent(X_train[:,0], X_train[:,1], y_train, epochs, loss_threshold)
        
    def predict(self, X_test):
        weighted_sum = self.w1 * X_test[:,0] + self.w2 * X_test[:,1] + self.bias
        y_pred = sigmoid(weighted_sum)
        return y_pred
    
    def evaluate(self, X_test, y_test):
        y_pred_proba = self.predict(X_test)
        y_pred = np.where(y_pred_proba > 0.5, 1, 0)  # Convert probabilities to binary predictions
        
        # Caculate confusion matrix
        true_positive = np.sum((y_pred == 1) & (y_test == 1))
        true_negative = np.sum((y_pred == 0) & (y_test == 0))
        false_positive = np.sum((y_pred == 1) & (y_test == 0))
        false_negative = np.sum((y_pred == 0) & (y_test == 1))

        # Make a table of confusion matrix
        print("Confusion Matrix:")
        print('-' * 55)
        print(f"| {'':^15} | {'Predicted 0':^15} | {'Predicted 1':^15} |")
        print('-' * 55)
        print(f"| {'Actual 0':^15} | {true_negative:^15} | {false_positive:^15} |")
        print(f"| {'Actual 1':^15} | {false_negative:^15} | {true_positive:^15} |")
        print('-' * 55)
        print()
        
        #
        precision = true_positive / (true_positive + false_positive)
        recall = true_positive / (true_positive + false_negative)
        f1_score = 2 * (precision * recall) / (precision + recall)
        
        accuracy = np.mean(y_pred == y_test)  # True = 1, False = 0
        # y_pred == y_test: returns an array containing boolean values [True False True ...]
        
        print('Accuracy: {}'.format(accuracy))
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1_score}")

## Train Model

In [9]:
model = myNN()
model.fit(X_train, y_train, epochs=10000, loss_threshold=0.411)

epochs=1    , w1=1                        , w2=1                        , bias=0                        , loss=0.7067871004847728       
epochs=2    , w1=0.9776620922402962       , w2=0.9480790207203986       , bias=-0.11226706670803967     , loss=0.6773703034021942       
epochs=3    , w1=0.9609125476331305       , w2=0.9052188491853423       , bias=-0.21016179153380088     , loss=0.6556794409382555       
epochs=4    , w1=0.9492783314404287       , w2=0.8707929230331629       , bias=-0.2949739823330707      , loss=0.6399398037987551       
epochs=5    , w1=0.9421933435116123       , w2=0.8439510422734104       , bias=-0.36819527558430154     , loss=0.6286123161559022       
epochs=6    , w1=0.9390600941197916       , w2=0.8237381332741572       , bias=-0.4313687285337374      , loss=0.6204572707469723       
epochs=7    , w1=0.9392964192684764       , w2=0.8091892305731584       , bias=-0.48597817221168843     , loss=0.614530223245117        
epochs=8    , w1=0.9423650744066836      

In [10]:
model.w1, model.w2, model.bias, model.loss

(7.595577471886923,
 1.2255342789155321,
 -3.785133925250397,
 0.41098189184875866)

## Predict Values

In [11]:
yp_test_proba = model.predict(X_test)
yp_test_proba

array([0.77524864, 0.86363738, 0.23283074, 0.27597245, 0.76173828,
       0.54104899])

In [12]:
yp_test = np.where(yp_test_proba > 0.5, 1, 0)
yp_test

array([1, 1, 0, 0, 1, 1])

In [13]:
y_test

array([1, 1, 0, 0, 1, 0], dtype=int64)

## Evaluate Model

In [14]:
model.evaluate(X_test,y_test)

Confusion Matrix:
-------------------------------------------------------
|                 |   Predicted 0   |   Predicted 1   |
-------------------------------------------------------
|    Actual 0     |        2        |        1        |
|    Actual 1     |        0        |        3        |
-------------------------------------------------------

Accuracy: 0.8333333333333334
Precision: 0.75
Recall: 1.0
F1 Score: 0.8571428571428571


### Examples of Making Boolean Array

In [15]:
accuracy = yp_test == y_test
accuracy

array([ True,  True,  True,  True,  True, False])

In [16]:
true_positive = (yp_test == 1) & (y_test == 1)
true_positive

array([ True,  True, False, False,  True, False])